2. PCA & t-SNE of Github features, classifying assets both by Marketcap and by Coin Function.
Given a dictionary of assets, _assets_, like in Github_analysis.ipynb:
For _feature_ in ['pr_count', 'issues_count', 'commit_add_sum', 'commit_del_sum', 'commit_count', 'star_count', 'return_pcnt', 'volatility_pcnt'] *# SEPARATE NOTEBOOK FOR EACH FEATURE*:
 
   a) create a dataframe, _feature_df_, of the values of _feature_ across all assets
   b) perform PCA on _feature_df_ and plot for both the 1st 2 principal components and 3 components
   c) perform t-SNE on _feature_df_ using 2 components and 3 components and plot each
   
Instead of the labels being the digit number like in the medium example, create labels based on Crypto100_labels.png.

Purpose: PCA and t-SNE are dimensionality reduction techniques, which help us understand how to visualize high-dimensional datasets.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob

## Create a dataframe, _feature_df_, of the values of _feature_ across all assets

In [5]:
_feature_ = 'pr_count'

In [11]:
path = 'github_data' 
allFiles = glob.glob(path + "/*.csv")
_feature_df_ = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0,usecols=['symbol',_feature_])
    list_.append(df)
_feature_df_ = pd.concat(list_)

In [12]:
print(_feature_df_)
#_feature_df_.head()

    symbol  pr_count
0      ADA         7
1      ADA         4
2      ADA         5
3      ADA        11
4      ADA         7
5      ADA         9
6      ADA         0
7      ADA         0
8      ADA         6
9      ADA         6
10     ADA         6
11     ADA         3
12     ADA         8
13     ADA         1
14     ADA         3
15     ADA         5
16     ADA         7
17     ADA         4
18     ADA         8
19     ADA         3
20     ADA         3
21     ADA         6
22     ADA         7
23     ADA         2
24     ADA         3
25     ADA         8
26     ADA         3
27     ADA         5
28     ADA         4
29     ADA         4
..     ...       ...
151    ZRX         0
152    ZRX         1
153    ZRX         3
154    ZRX         0
155    ZRX         3
156    ZRX         3
157    ZRX         0
158    ZRX         0
159    ZRX         2
160    ZRX         4
161    ZRX         2
162    ZRX         0
163    ZRX         1
164    ZRX         0
165    ZRX         0
166    ZRX   

## Perform PCA on _feature_df_ and plot for both the 1st 2 principal components and 3 components